In [1]:
import numpy as np
import pandas as pd
import math_functions as mf

In [2]:
# set random seed
np.random.seed(1234)

In [92]:
# generate random Logistic growth curves
n = 20 # number of curves to generate
max_time_points_growth_curve = 1000 # maximum number of time points in the growth curve
t_max = 200
time_points_range = [np.around(x, 3) for x in np.linspace(0, t_max, max_time_points_growth_curve).tolist()]

out = pd.DataFrame(columns=['A', 'mu_max', 'l', 'n_timepoints', 't [h]'] + time_points_range)

for i in range(n):
    print(i)
    # generate random parameters
    A = np.random.uniform(0, 20)
    mu_max = np.random.uniform(0, 2)
    l = np.random.uniform(0, 100)
    print(A, mu_max, l)

    # generate random time points between 0 and 200
    n_time_points = np.random.randint(15, 1000) # number of time points in the growth curve
    n_time_points = 1000
    t = np.random.choice(time_points_range, n_time_points, replace=False)

    t.sort()

    # generate growth curve
    y = mf.modified_logistic(t, A, mu_max, l)

    # add noise to the growth curve
    y = [y_i + np.random.normal(0, 0.1) for y_i in y]


    out_i = pd.Series([A, mu_max, l, n_time_points, 'sample {}'.format(i)] + y, index=['A', 'mu_max', 'l', 'n_timepoints', 't [h]'] + t.tolist())
    out = pd.concat([out, out_i.to_frame().T])



out = out.reset_index(drop=True)
out.to_excel('./synthetic_logistic_growth_curves.xlsx', index=False)
out_clean = out.iloc[:, 4:]
# out_clean.columns[1:] = out_clean.columns[1:].astype(float)
out_clean.to_excel('./synthetic_logistic_growth_curves_dgc_ready.xlsx', index=False)  # drop the first 4 columns


0
2.6115334498274234 1.0648118967098252 88.43408944468115
1
9.46081324744658 0.18937976829426661 32.37610260260446
2
10.411662398024088 1.5764328381473587 50.092811287027075
3
10.697751730658036 0.9612558940034548 20.120646997466775
4
7.3353960638811095 1.2737987805484996 67.22945160817211
5
19.33888658415066 1.0771841194230765 9.332699201622008
6
19.189966463572247 0.3837720513313323 31.06671374636807
7
6.4099999535130525 0.0968614729450974 43.23733581551971
8
19.377738231037405 1.8751920686438441 80.81475631811314
9
17.86012225416152 1.5205323966846256 57.22765356212142
10
17.357329254060016 0.4419904356292217 83.81021283697811
11
7.268850691961786 0.40632483348579007 82.7923820816022
12
12.302983982027161 0.4965524394242451 45.340331104272614
13
15.153834068751816 0.05893626595440904 53.944971206260064
14
7.519170913221478 1.705262523269751 72.32657617277071
15
9.893188914104893 1.6944371045825264 71.71743779297111
16
0.44677016184812235 1.9444290063999796 92.87342778074539
17
2.701

/home/m/Desktop/dashing_growth_curves/growthdash/implementation_test/math_functions.py:29: RuntimeWarning:

overflow encountered in power



In [66]:
out_clean.mean(axis=1)

TypeError: can only concatenate str (not "int") to str

In [61]:
pd.read_excel('./synthetic_logistic_growth_curves_dgc_ready.xlsx')['t [h]']


0    0
Name: t [h], dtype: int64

In [70]:
import plotly.graph_objects as go
out_temp = out.iloc[:, 5:] # drop the first 4 columns
# drop all columns with nan values
out_temp = out_temp.loc[:, out_temp.isna().sum(axis=0) != 2]
out_temp.columns = out_temp.columns.astype(float)
data = [go.Scatter(x=out_temp.columns, y=out_temp.values[0])]
fig = go.Figure(data=data)
fig

In [89]:
np.nanmean(out_temp.values, axis=0)
np.nanstd(out_temp.values.astype(float), axis=0)

array([0., 0., 0., ..., 0., 0., 0.])

In [88]:
np.nanstd(out_temp.values[:, :2].astype(float), axis=0)

array([0., 0.])

In [68]:
# generate random Logistic growth curves
n = 100 # number of curves to generate
max_time_points_growth_curve = 10000 # maximum number of time points in the growth curve
t_max = 200
time_points_range = [np.around(x, 3) for x in np.linspace(0, t_max, max_time_points_growth_curve).tolist()]

out = pd.DataFrame(columns=['A', 'mu_max', 'l', 'n_timepoints'] + time_points_range)

for i in range(n):
    print(i)
    # generate random parameters
    A = np.random.uniform(0, 20)
    mu_max = np.random.uniform(0, 2)
    l = np.random.uniform(0, 100)

    # generate random time points between 0 and 200
    n_time_points = np.random.randint(15, 1000) # number of time points in the growth curve
    t = np.random.choice(time_points_range, n_time_points, replace=False)

    t.sort()

    # generate growth curve
    y = mf.modified_gompertz(t, A, mu_max, l)

    # add noise to the growth curve
    y = [y_i + np.random.normal(y_i, 0.1 * y_i) for y_i in y]

    out_i = pd.Series([A, mu_max, l, n_time_points] + y, index=['A', 'mu_max', 'l', 'n_timepoints'] + t.tolist())
    out = pd.concat([out, out_i.to_frame().T])

    # out[i] = {'A': A, 'mu_max': mu_max, 'l': l, 't': t, 'y': y}

out = out.reset_index(drop=True)
out.to_excel('./synthetic_gompertz_growth_curves.xlsx')
out.iloc[:, 4:].to_excel('./synthetic_gompertz_growth_curves_dgc_ready.xlsx')  # drop the first 4 columns

0


C:\Users\mreiter\AppData\Local\Temp\ipykernel_17864\2386029802.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat([out, out_i.to_frame().T])
C:\Users\mreiter\AppData\Local\Temp\ipykernel_17864\2386029802.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat([out, out_i.to_frame().T])


1
2


C:\Users\mreiter\AppData\Local\Temp\ipykernel_17864\2386029802.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat([out, out_i.to_frame().T])


3


...

96
97


C:\Users\mreiter\AppData\Local\Temp\ipykernel_17864\2386029802.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat([out, out_i.to_frame().T])


98


C:\Users\mreiter\AppData\Local\Temp\ipykernel_17864\2386029802.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat([out, out_i.to_frame().T])


99


In [69]:
x = pd.concat([out, out_i.to_frame().T])
x = x.loc[:, ~x.iloc[0, :].isnull()]
x

,A,mu_max,l,n_timepoints,0.28,0.34,0.36,0.44,0.84,0.86,...,196.0,196.88,197.14,197.4,197.56,198.34,198.36,198.52,199.3,199.82
0,14.617287,1.582383,88.724869,392.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,29.235314,27.737606,29.798819,27.011125,27.529202,25.067168,27.194309,29.844718,26.83695,29.827768
1,14.031728,0.349390,44.409168,786.0,NaN,NaN,NaN,NaN,8.534506e-22,NaN,...,NaN,NaN,NaN,NaN,28.537352,NaN,NaN,26.305207,NaN,NaN
2,15.148849,0.495832,64.523725,16.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.349973,0.584823,97.097219,138.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17.968926,1.914455,44.787692,303.0,NaN,NaN,NaN,NaN,NaN,NaN,...,36.487035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,10.408128,1.407750,99.867271,30.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,21.422786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,6.098110,1.919811,46.892503,612.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.455049,NaN,NaN
98,3.201499,1.951015,32.307401,565.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.360604,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,9.967405,1.817842,66.465240,311.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.835133,NaN,NaN,NaN


In [70]:
x.iloc[0, :].isnull()

A               False
mu_max          False
l               False
n_timepoints    False
0.28            False
                ...  
198.34          False
198.36          False
198.52          False
199.3           False
199.82          False
Name: 0, Length: 396, dtype: bool